In [1]:
%load_ext rpy2.ipython

# 扩增子测序分析结果报告

微生物多样性测序是一种利用高通量测序技术对PCR所扩增的16S、18S、ITS等微生物物种特征序列进行检测的研究方法。 根据测序reads对目标扩增区域的覆盖情况，来统计丰度。原核生物常用16s引物，有v3-v4，v4等，不同的研究目的适用不同的引物。
> 真菌常用ITS，即核糖体基因转录间隔区（internal transcribed spacer ,ITS），是位于核糖体DNA（rDNA）上18S 和28S 基因之间的区域片段，主要包括ITS1和ITS2。常用于真菌多样性分析。

## 实验流程

从样本中提取基因组DNA后，用带有barcode的特异引物扩增rDNA的保守区（引物序列信息如表）。

| 类型     | 测序区域             | 正向引物名称  | 引物序列            | 产物长度 | 正向引物名称  | 引物序列            
| -------- | -------------------- | --------- | ------------------- | -------- |------------------- | -------- |
| 16S 细菌 | V3-V4                | 341F      | CCTACGGGNGGCWGCAG   | ~466     | 806R     | GGACTACHVGGGTATCTAAT |
| 16S 细菌 | V4                   | 515F      | GTGYCAGCMGCCGCGGTAA | ~292     | 806R     | GGACTACNVGGGTWTCTAAT |
| 16S 细菌 | V5-V7                | 799F      | AACMGGATTAGATACCCKG | ~414     | 1193R    | ACGTCATCCCCACCTTCC ｜
| 16S 细菌 | V4-V5                | 515F      | GTGCCAGCMGCCGCGGTAA | ~393     |907R     | CCGTCAATTCCTTTGAGTTT |
| 16S 古菌 | V4-V5                | Arch519F  | CAGCMGCCGCGGTAA     | ~397     | Arch915R | GTGCTCCCCCGCCAATTCCT |
| 18S      | V4                   | 528F      | GCGGTAATTCCAGCTCCAA | ~179     | 706R     | AATCCRAGAATTTCACCTCT |
| ITS      | ITS2                 | ITS3_KYO2 | GATGAAGAACGYAGYRAA  | ~381     |ITS4     | TCCTCCGCTTATTGATATGC |

## 分析流程

微生物多样性研究主要分为多样性研究、功能研究、样本差异比较和环境关系研究5大部分。 测序得到raw reads之后，我们对低质量reads进行过滤，之后采用DADA2软件[1]进行质控、聚类单碱基精度的ASV (Amplicon Sequence Variants)，相当于100%相似度聚类的OTU。DADA2（Divisive Amplicon Denoising Algorithm）利用overlap将双端数据进行拼接，并进行质控、嵌合体过滤，获得高质量的ASV。在此基础上进行丰度定量，差异分析，以及物种注释和功能基因分析等。

In [2]:
import qiime2 as q2
import os 
import subprocess
import pandas as pd

正式分析开始------------

## 数据处理

### 创建文件目录

In [3]:
if not os.path.exists("qiime2_result"):
    os.mkdir("qiime2_result")
if not "qiime2_result" in os.getcwd():
    os.chdir("qiime2_result")
# set the output dir-----------------
dir1 = "01.DataImport_QC_Filter"
dir2 = "02.Denoise_Cluster"
dir3 = "03.TaxonomyClassify"
dir4 = "04.Taxa_diff"
dir5 = "05.Alpha_diversity"
dir6 = "06.beta_diversity"
dir7 = "07.Function"

In [4]:
#which version of qiime2 and python are we using?
!qiime info > env.description.txt

In [6]:
for v in globals().keys():
    if v.startswith('dir') and not os.path.exists(globals()[v]):
        os.mkdir(globals()[v])

### 数据导入

必须的文件路径参数
1. fq_file: 输入的fq文件的路径，三列， 第一列`sample-id`是样本名称，第二列`forward-absolute-filepath`是read1的路径，第三列`reverse-absolute-filepath`是read2的路径，制表符分割的TSV文件
2. metadata_file: 样本分组信息，可以多列，但第一列必须是与`fq_file`的第一列`sample-id`一致
3. classify_file: 训练好的分类器，可以去网站上[下载](https://docs.qiime2.org/2022.2/data-resources/)

In [7]:
fq_file = '/Users/dalena/Data/meta/example_rawdata/filepath.manifest1'
metadata_file = '/Users/dalena/Data/meta/example_rawdata/sample-metadata.tsv'
classify_file = '/Users/dalena/Data/meta/ref_Data/silva-138-99-515-806-nb-classifier.qza'

In [8]:
# 生成保存qiime2结果的主文件夹
if not os.path.exists("qiime2_core_output"):
    os.mkdir("qiime2_core_output")
if not "qiime2_core_output" in os.getcwd():
    os.chdir("qiime2_core_output")

In [9]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path {fq_file} \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path import_data.qza

Imported /Users/dalena/Data/meta/example_rawdata/filepath.manifest1 as PairedEndFastqManifestPhred33V2 to import_data.qza


### 数据质量

导入数据成功后，需要观察数据中的数据质量，是否有接头，和两端低质量的碱基，根据碱基的质量值进行过滤

In [10]:
!qiime demux summarize \
  --i-data ./import_data.qza \
  --o-visualization ./qc_visualizers.qzv
q2.Visualization.load('qc_visualizers.qzv')

Saved Visualization to: ./qc_visualizers.qzv


<visualization: Visualization uuid: 22850167-2416-44b5-b76c-fe2a26cff0a1>

### 去引物接头

因为扩增子测序依赖于特定的引物PCR产物，所以测序的reads可能存在少许引物序列，好在引物都是固定的，可以查[实验流程章节](#)

In [13]:
#-------parameter 输出----------
TRIMADAPTERS=True # 是否需要剪切接头
Fprimer="GTGCCAGCMGCCGCGGTAA" #515F, replace if different primer
Rprimer="GGACTACHVGGGTWTCTAAT" #806R, replace if different primer
Ncore=4 #多核使用

In [14]:
#------------------------------
if TRIMADAPTERS:
  !qiime cutadapt trim-paired \
    --i-demultiplexed-sequences import_data.qza \
    --p-cores {Ncore} \
    --p-front-f {Fprimer} \
    --p-front-r {Rprimer} \
    --p-match-adapter-wildcards \
    --o-trimmed-sequences import_data_removeAdapter.qza

Saved SampleData[PairedEndSequencesWithQuality] to: import_data_removeAdapter.qza


In [15]:
!qiime demux summarize \
  --i-data import_data_removeAdapter.qza \
  --o-visualization import_data_removeAdapter.qzv
q2.Visualization.load('import_data_removeAdapter.qzv')

Saved Visualization to: import_data_removeAdapter.qzv


<visualization: Visualization uuid: 179dd440-3422-445e-b582-f3cd16d184cc>

但其实好像没啥影响，不过这个步骤可以固定，有就去除, 没有就万事大吉。

### 质控拼接
测序得到原始数据后，由于PCR错误、测序错误等会产生大量的低质量数据或者无生物学意义数据（例如嵌合体）， 因此为保证后续分析具有统计可靠性和生物学有效性，因此必要的质控是必要的，需要去除：

- Reads剪裁；
- 低质量Reads过滤；
- 基于错误率模型序列校正；
- Tags去嵌合体。

#### Reads剪裁

根据上级分析结果，去除接头的数据`import_data_removeAdapter.qza`,根据可视化的界面查看低质量的碱基，这样可以用来制定修剪的长度和位置。

In [16]:
# 根据输入数据的质量控制的结果，选择输出reads的去除长度
FR_cut_tail=200 #正向reads的尾巴切割位置，equivalent to p-trunc-len-f
RR_cut_tail=150 #反向reads的尾巴切割位置，equivalent to p-trunc-len-r
FR_cut_head=13 #正向reads的前端切割位置，equivalent to trim-left-f, updated to 0 as primers already stripped
RR_cut_head=13 #反向reads的前端切割位置，equivalent to trim-left-r, updated to 0 as primers already stripped
Ncore=4

In [17]:
if not os.path.exists("denoise_stat.qza"):
    !qiime dada2 denoise-paired \
        --i-demultiplexed-seqs import_data_removeAdapter.qza \
        --p-trunc-len-f {FR_cut_tail} \
        --p-trunc-len-r {RR_cut_tail} \
        --p-trim-left-f {FR_cut_head} \
        --p-trim-left-r {RR_cut_head} \
        --p-n-threads {Ncore} \
        --output-dir dada2_denoise_out \
        --o-table table.qza \
        --o-representative-sequences rep-seqs.qza \
        --o-denoising-stats denoise_stat.qza 
else:
    print("Dada2 as already complete")

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoise_stat.qza


### 质量统计
对各样本过滤前后reads、tag信息统计汇总如下

In [18]:
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file {metadata_file}
q2.Visualization.load('table.qzv')

Saved Visualization to: table.qzv


<visualization: Visualization uuid: 482aff1c-a448-461a-96b5-330c46ab1d78>

In [19]:
!qiime tools export --input-path table.qzv --output-path ../{dir2}/_Tags_Stat
!qiime tools export --input-path table.qza --output-path ../{dir2}
!biom convert -i ../{dir2}/feature-table.biom -o ../{dir2}/feature-table.tsv --to-tsv

Exported table.qzv as Visualization to directory ../02.Denoise_Cluster/_Tags_Stat
Exported table.qza as BIOMV210DirFmt to directory ../02.Denoise_Cluster


In [20]:
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv
q2.Visualization.load('rep-seqs.qzv')

Saved Visualization to: rep-seqs.qzv


<visualization: Visualization uuid: 6a7b2f60-9831-4fec-99b7-b698d9f304ae>

In [21]:
!qiime tools export --input-path rep-seqs.qzv --output-path ../{dir2}/_Tags_Seq

Exported rep-seqs.qzv as Visualization to directory ../02.Denoise_Cluster/_Tags_Seq


## 质控统计

In [22]:
!qiime metadata tabulate \
  --m-input-file denoise_stat.qza \
  --o-visualization denoise_stat.qzv
q2.Visualization.load('denoise_stat.qzv')

Saved Visualization to: denoise_stat.qzv


<visualization: Visualization uuid: 20ac1f14-71e7-48df-b2a5-2c6c6ccd41bd>

In [23]:
!qiime tools export --input-path='denoise_stat.qzv' --output-path=../{dir1}/_Denoise_stat

Exported denoise_stat.qzv as Visualization to directory ../01.DataImport_QC_Filter/_Denoise_stat


## ASV分析

使用DADA2质控后产生的每个去重的序列称为ASVs (amplicon sequence variants)， 或称为特征序列(对应于OTU代表序列)，而这些序列在样本中的丰度表称为特征表(对应等同于OTU表)。 以DADA2为代表的去噪生成特征序列的方法是目前主流分析平台(QIIME2和USEARCH)所力推的。获得ASV/OTU之后，利用相关软件，根据其丰度和序列信息，能够逐一开展物种注释、群落多样性、组间差异等多种核心分析。

## 物种注释
微生物物种分类一般分为界、门、纲、目、科、属、种7个等级，而每个ASV代表某类型分类水平集合。因此根据ASV的序列信息进行物种注释，能将分析结果与实际的生物学意义进行关联，从而研究群落中物种的变化关系等内容。

物种注释方法：DADA2在构建ASVs的过程中会选取代表性序列（ASV中丰度最高的那条Tag序列），将这些代表性序列集合用RDP Classifier的Naïve Bayesian assignment算法，与选定的数据库进行物种注释（设定置信度的阈值为0.8~1）。

需要先从网站上下载分类模型，网站[链接](https://docs.qiime2.org/2022.2/data-resources/),并给出路径`{classify_file}`。

In [24]:
#Use machine learning to identify your representative sequences against an external database
#This step may take a few minutes
if not os.path.exists('taxonomy.qza'):
    !qiime feature-classifier classify-sklearn \
      --i-classifier {classify_file} \
      --i-reads rep-seqs.qza \
      --o-classification taxonomy.qza
else:
    print("Taxonomy Classify have done!")

Saved FeatureData[Taxonomy] to: taxonomy.qza


In [26]:
!qiime tools export --input-path=taxonomy.qza --output-path=../{dir3}

Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory ../03.TaxonomyClassify


In [27]:
#View the taxonomic designations
!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved Visualization to: taxonomy.qzv


### 微生物表达总表

- ASV的表达总表在[02.Denoise_Cluster/feature-table.tsv](02.Denoise_Cluster/feature-table.tsv)
- ASV的注释总表在[03.TaxonomyClassify/taxonomy.tsv ](03.TaxonomyClassify/taxonomy.tsv)
- ASV的序列在[02.Denoise_Cluster/_Tags_Seq/sequences.fasta](02.Denoise_Cluster/_Tags_Seq/sequences.fasta)

In [28]:
import pysam
with pysam.FastxFile("../"+dir2+"/_Tags_Seq/sequences.fasta") as fa, open("../"+dir3+"/ASV_sequences.tsv","w") as out:
    for read in fa:
        out.write(read.name+"\t"+read.sequence+"\n")

In [29]:
%%R -i dir2  -i dir3
require(tidyverse)
data_exp = read_tsv(paste("../",{dir2},"/feature-table.tsv",sep = ""),comment = "# ",show_col_types = FALSE)
data_taxa = read_tsv(paste("../",{dir3},"/taxonomy.tsv",sep = ""),show_col_types = FALSE)
data_seq = read_tsv(paste("../",{dir3},"/ASV_sequences.tsv",sep = ""),col_names = F,show_col_types = FALSE)
colnames(data_exp)[1] = colnames(data_taxa)[1]
colnames(data_seq) = c(colnames(data_taxa)[1],"sequence")
tmp = left_join(data_exp,data_taxa,by  = colnames(data_taxa)[1])
final_data = left_join(tmp,data_seq,by  = colnames(data_taxa)[1])
write_tsv(final_data,paste("../",{dir3},"/Taxonomy_exp_seq.tsv",sep = ""))

R[write to console]: 载入需要的程辑包：tidyverse

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



最终展现的的统计表格如下：

In [30]:
pd.read_csv("../"+dir3+"/Taxonomy_exp_seq.tsv",sep = "\t")

,Feature ID,F3D0,F3D1,F3D141,F3D142,F3D143,F3D144,F3D145,F3D146,F3D147,...,F3D2,F3D3,F3D5,F3D6,F3D7,F3D8,F3D9,Taxon,Confidence,sequence
0,40394bfe4a2f991e0651e5a311f3ee24,593,426,453,298,238,439,656,323,1543,...,3592,1009,328,1040,662,287,524,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.999999,AGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGAAGA...
1,a653d2e8c495970f57c1fc1d8d5a3eb8,351,357,369,307,177,288,501,235,1244,...,1622,615,271,686,508,355,433,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.905046,AGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGCAGGCGGACTC...
2,d43ed378a846746647619025dcb17b10,455,235,338,157,211,302,535,263,931,...,1204,484,281,597,449,353,490,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.980320,AGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGGCTG...
3,e176441fb5064973ee3a5222838d750b,449,75,522,170,236,365,602,407,1126,...,479,208,165,411,319,151,215,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,0.996699,AGCGTTATCCGGATTTATTGGGTTTAAAGGGTGCGTAGGCGGGCTT...
4,c27f7abd3abc4f5086d6019c9ef2e0d5,178,142,222,223,135,153,400,216,594,...,392,480,154,550,566,603,666,d__Bacteria; p__Bacteroidota; c__Bacteroidia; ...,1.000000,AGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGATTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,6843c27a57bd50b0a45229b64bade573,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,d__Bacteria; p__Firmicutes; c__Clostridia; o__...,0.986823,AGCGTTGTCCGGAATGACTGGGCGTAAAGGGAGTGTAGGCGGCCTT...
216,cb80b54178ea1877ec0929b98f72ec69,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,d__Bacteria; p__Actinobacteriota; c__Coriobact...,0.964198,AGCGTTATCCGGATTCATTGGGCGTAAAGCGCGCGTAGGCGGCCTG...
217,40601bb335e898d24650af8d1221c7cc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,d__Bacteria; p__Firmicutes; c__Bacilli; o__RF3...,0.993946,AGCGTTATCCGGATTTATTGGGCGTAAAGCGTGCGCAGGCGGTTTG...
218,2e60a2b63ef994fe9a9f8de5efe538c2,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,d__Bacteria; p__Firmicutes; c__Bacilli; o__Lac...,1.000000,AGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTTG...


可以通过预览物种堆叠图快速了解微生物的在样本中的组成

In [31]:
#View the taxonomic composition of each sample in an interactive bar plot
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file  {metadata_file} \
  --o-visualization taxa-bar-plots.qzv
q2.Visualization.load('taxa-bar-plots.qzv')

Saved Visualization to: taxa-bar-plots.qzv


<visualization: Visualization uuid: 323222eb-362a-4a35-bc5b-189256a29133>

根据ASV的物种注释信息，统计每个样品在各个分类水平（界门纲目科属种）上的Tags序列数目， 并绘制柱状图，详细信息如下所示。

## 菌种距离矩阵分析

在计算样本在微生物群落的多样性的时候，除了需要菌群表达信息，还需要考虑ASV之间的遗传进化关系，同时根据考虑ASV丰度的区别， 常用的距离计算指标有：
Unweighted-Unifrac、Weighted-Unifrac、Bray–Curtis以及Jaccard指数等。

根据有没有考虑ASV丰度的区别， 可分为加权（Weighted Unifrac）和非加权（Unweighted Unifrac）两种方法。 其中Unweighted UniFrac只考虑了物种有无的变化，而Weighted UniFrac则同时考虑物种有无和物种丰度的变化。 因此在实际分析中，结合两种Unifrac分析方法，能更有效发现样本之间的结构差异信息。

In [32]:
# Step5 phylogeny tree and distance matrix
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza


## Alpha多样性分析
α多样性是指特定生境或者生态系统内的多样性情况，它可以指示生境被物种隔离的程度，通常利用物种丰富度（种类情况） 与物种均匀度（分布情况）两个重要参数来计算。指标很多，qiime2只默认计算稀释曲线，faith_pd，shannon，evenness，observed这几种。

In [33]:
# Step6 alpha diversity analysis
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 2000 \
  --m-metadata-file {metadata_file} \
  --output-dir core-metrics-results

Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/bray_curtis_pcoa_re

In [34]:
#!qiime tools export --input-path=core-metrics-results/rarefied_table.qza --output-path=../{dir5}/rarefied
!qiime tools export --input-path=core-metrics-results/faith_pd_vector.qza --output-path=../{dir5}/faith_pd
!qiime tools export --input-path=core-metrics-results/observed_features_vector.qza --output-path=../{dir5}/observed
!qiime tools export --input-path=core-metrics-results/evenness_vector.qza --output-path=../{dir5}/evenness
!qiime tools export --input-path=core-metrics-results/shannon_vector.qza --output-path=../{dir5}/shannon

Exported core-metrics-results/faith_pd_vector.qza as AlphaDiversityDirectoryFormat to directory ../05.Alpha_diversity/faith_pd
Exported core-metrics-results/observed_features_vector.qza as AlphaDiversityDirectoryFormat to directory ../05.Alpha_diversity/observed
Exported core-metrics-results/evenness_vector.qza as AlphaDiversityDirectoryFormat to directory ../05.Alpha_diversity/evenness
Exported core-metrics-results/shannon_vector.qza as AlphaDiversityDirectoryFormat to directory ../05.Alpha_diversity/shannon


In [35]:
%%R -i dir5 -i metadata_file
data = read.table(metadata_file,row.names = 1,header = T)
for(path in list.files(list.dirs(paste("../",dir5,sep = "")),pattern = "*.tsv",full.names = TRUE)){
    tmp = read.table(path,row.names = 1,header = T)
    data = cbind(data,tmp)
}
write.csv(data,paste("../",dir5,"/summary_alpha_diversity.csv",sep = ""))

In [36]:
pd.read_csv("../"+dir5+"/summary_alpha_diversity.csv")

,Unnamed: 0,time,dpw,pielou_evenness,faith_pd,observed_features,shannon_entropy
0,F3D0,Early,0,0.839872,6.848429,98,5.555509
1,F3D1,Early,1,0.869344,6.902300,99,5.763192
2,F3D141,Late,141,0.790978,6.223620,77,4.956888
3,F3D142,Late,142,0.805085,5.488893,50,4.543787
4,F3D143,Late,143,0.817669,5.134559,59,4.810057
5,F3D144,Late,144,0.770699,5.456914,55,4.455691
6,F3D145,Late,145,0.737855,6.381842,68,4.491663
7,F3D146,Late,146,0.818636,6.089748,85,5.246960
8,F3D147,Late,147,0.735346,6.971328,97,4.853217
9,F3D148,Late,148,0.752291,7.112360,94,4.930959


### 可视化

In [37]:
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 4000 \
  --m-metadata-file {metadata_file} \
  --o-visualization core-metrics-results/alpha-rarefaction.qzv

q2.Visualization.load('core-metrics-results/alpha-rarefaction.qzv')

Saved Visualization to: core-metrics-results/alpha-rarefaction.qzv


<visualization: Visualization uuid: 97e7f3df-df6f-43ce-b355-22a39f44b8df>

### 多样性差异

In [38]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file {metadata_file} \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

q2.Visualization.load('core-metrics-results/evenness-group-significance.qzv')

Saved Visualization to: core-metrics-results/evenness-group-significance.qzv


<visualization: Visualization uuid: d79c38e3-b3fe-4b01-9882-c2146ea9dedd>

In [39]:
#Using group significance can show you how different factors in your metadata affect your diversity

!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file  {metadata_file} \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

q2.Visualization.load('core-metrics-results/faith-pd-group-significance.qzv')

Saved Visualization to: core-metrics-results/faith-pd-group-significance.qzv


<visualization: Visualization uuid: c3398e11-375a-46ae-a1e4-f8a2be2fbad5>

## Beat多样性
Beta多样性是不同生态系统之间多样性的比较，是物种组成沿环境梯度或者在群落间的变化率， 用来表示生物种类对环境异质性的反应。一般来说，不同环境梯度下群落Beta多样性计算包括物种改变（多少） 和物种产生（有无）两部分。因此，我们将根据这两个重要指标，运用Weighted Unifrac、Unweighted Unifrac、Jaccard 以及 Bray四个个指数进行后续Beta多样性分析。

In [40]:
# bray_curtis
!qiime tools export --input-path=core-metrics-results/bray_curtis_emperor.qzv --output-path=../{dir6}/bray_curtis
!qiime tools export --input-path=core-metrics-results/bray_curtis_pcoa_results.qza --output-path=../{dir6}/bray_curtis
!qiime tools export --input-path=core-metrics-results/bray_curtis_distance_matrix.qza --output-path=../{dir6}/bray_curtis

# jaccard
!qiime tools export --input-path=core-metrics-results/jaccard_emperor.qzv --output-path=../{dir6}/jaccard
!qiime tools export --input-path=core-metrics-results/jaccard_pcoa_results.qza --output-path=../{dir6}/jaccard
!qiime tools export --input-path=core-metrics-results/jaccard_distance_matrix.qza --output-path=../{dir6}/jaccard

# weighted_unifrac
!qiime tools export --input-path=core-metrics-results/weighted_unifrac_emperor.qzv --output-path=../{dir6}/weighted_unifrac
!qiime tools export --input-path=core-metrics-results/weighted_unifrac_pcoa_results.qza --output-path=../{dir6}/weighted_unifrac
!qiime tools export --input-path=core-metrics-results/weighted_unifrac_distance_matrix.qza --output-path=../{dir6}/weighted_unifrac

# weighted_unifrac
!qiime tools export --input-path=core-metrics-results/unweighted_unifrac_emperor.qzv --output-path=../{dir6}/unweighted_unifrac
!qiime tools export --input-path=core-metrics-results/unweighted_unifrac_pcoa_results.qza --output-path=../{dir6}/unweighted_unifrac
!qiime tools export --input-path=core-metrics-results/unweighted_unifrac_distance_matrix.qza --output-path=../{dir6}/unweighted_unifrac

Exported core-metrics-results/bray_curtis_emperor.qzv as Visualization to directory ../06.beta_diversity/bray_curtis
Exported core-metrics-results/bray_curtis_pcoa_results.qza as OrdinationDirectoryFormat to directory ../06.beta_diversity/bray_curtis
Exported core-metrics-results/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory ../06.beta_diversity/bray_curtis
Exported core-metrics-results/jaccard_emperor.qzv as Visualization to directory ../06.beta_diversity/jaccard
Exported core-metrics-results/jaccard_pcoa_results.qza as OrdinationDirectoryFormat to directory ../06.beta_diversity/jaccard
Exported core-metrics-results/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory ../06.beta_diversity/jaccard
Exported core-metrics-results/weighted_unifrac_emperor.qzv as Visualization to directory ../06.beta_diversity/weighted_unifrac
Exported core-metrics-results/weighted_unifrac_pcoa_results.qza as OrdinationDirectoryFormat to directory ../06.b

### 可视化展示
散点图展示`beta`多样性

In [41]:
#View unweighted unifrac beta diversity on a PCoA emperor plot
q2.Visualization.load('core-metrics-results/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 63bb76f0-0389-44a5-808a-76c14824e07d>

如果想要更改绘图的坐标，需要重新绘制

In [42]:
# putting cusom axes on your PCoA emperor plot (shown above)
!qiime emperor plot \
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file {metadata_file} \
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv
q2.Visualization.load('core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv')

Saved Visualization to: core-metrics-results/unweighted-unifrac-emperor-days-since-experiment-start.qzv


<visualization: Visualization uuid: f58351e0-7304-4b96-be3c-f28f96e69b40>

### 多样性差异

In [43]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file  {metadata_file} \
  --m-metadata-column time \
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv \
  --p-pairwise

q2.Visualization.load('core-metrics-results/unweighted-unifrac-body-site-significance.qzv')

Saved Visualization to: core-metrics-results/unweighted-unifrac-body-site-significance.qzv


<visualization: Visualization uuid: e6122689-8a44-469b-8da3-56d7d9539016>

## 差异分析

In [44]:
# 生成标准化后的微生物丰度矩阵
!qiime composition add-pseudocount \
  --i-table table.qza \
  --o-composition-table comp-table.qza

Saved FeatureTable[Composition] to: comp-table.qza


In [45]:
!qiime tools export --input-path=comp-table.qza --output-path=../{dir4}
!biom convert -i ../{dir4}/feature-table.biom -o ../{dir4}/comp_feature-table.tsv --to-tsv

Exported comp-table.qza as BIOMV210DirFmt to directory ../04.Taxa_diff


In [46]:
# ANCOM operates on a FeatureTable[Composition] QIIME 2 artifact, which is based on frequencies of features on a per-sample basis, but cannot tolerate frequencies of zero. 
# To build the composition artifact, a FeatureTable[Frequency] artifact must be provided to add-pseudocount (an imputation method), which will produce the FeatureTable[Composition] artifact.

!qiime composition ancom \
  --i-table comp-table.qza \
  --m-metadata-file {metadata_file} \
  --m-metadata-column time \
  --o-visualization ancom-subject.qzv
q2.Visualization.load('ancom-subject.qzv')

Saved Visualization to: ancom-subject.qzv


<visualization: Visualization uuid: a66af46b-fd76-4900-a07d-a40b6719583f>

In [47]:
#To look at differential abundance at a different taxonomic level
!qiime taxa collapse \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 5 \
  --o-collapsed-table table-l5.qza

Saved FeatureTable[Frequency] to: table-l5.qza


In [48]:
# Redo the add-pseudocount on the new table

!qiime composition add-pseudocount \
  --i-table table-l5.qza \
  --o-composition-table comp-table-l5.qza

Saved FeatureTable[Composition] to: comp-table-l5.qza


In [50]:
#Run the analysis on the collapsed taxonomy

!qiime composition ancom \
  --i-table comp-table-l5.qza \
  --m-metadata-file {metadata_file} \
  --m-metadata-column time \
  --o-visualization l5-ancom-subject.qzv


q2.Visualization.load('l5-ancom-Subject.qzv')

Saved Visualization to: l5-ancom-subject.qzv


<visualization: Visualization uuid: 90e28777-cc13-489f-8c63-46d63309e50d>

## PICRUST2 功能

In [211]:
!qiime picrust2 full-pipeline \
   --i-table table.qza \
   --i-seq rep-seqs.qza \
   --output-dir q2-picrust2_output \
   --p-threads 4 \
   --p-max-nsti 2 

Error: QIIME 2 has no plugin/command named 'picrust2'.
